In [1]:
import torch as th
from mamkit.models.audio import BiLSTM  # Asegurate que este sea el path correcto
from mamkit.configs.audio import BiLSTMMFCCsConfig
from mamkit.configs.base import ConfigKey
from mamkit.data.datasets import InputMode

# Definir la clave de configuración
config_key = ConfigKey(
    dataset='mmused-fallacy',
    task_name='afc',
    input_mode=InputMode.AUDIO_ONLY,
    tags={'anonymous'}
)

# Cargar la configuración usando el mapeo que ya está definido en BiLSTMMFCCsConfig
config = BiLSTMMFCCsConfig.from_config(key=config_key)

# Crear el modelo
model = BiLSTM(
    embedding_dim=config.embedding_dim,
    lstm_weights=config.lstm_weights,
    head=config.head,
    dropout_rate=config.dropout_rate
)

# Mover a GPU si está disponible
device = th.device("cuda" if th.cuda.is_available() else "cpu")
model = model.to(device)

In [2]:
import pandas as pd

train_df = pd.read_csv("train_afc_audio.csv")



print(train_df.head())

                                                Ruta  Etiqueta
0  [WindowsPath('C:/Users/Usuario/MMUSED-fallacy/...         0
1  [WindowsPath('C:/Users/Usuario/MMUSED-fallacy/...         0
2  [WindowsPath('C:/Users/Usuario/MMUSED-fallacy/...         1
3  [WindowsPath('C:/Users/Usuario/MMUSED-fallacy/...         0
4  [WindowsPath('C:/Users/Usuario/MMUSED-fallacy/...         0


In [ ]:
import torch
from torch.utils.data import Dataset
import torchaudio
import re  # Para usar expresiones regulares

class AudioDataset(Dataset):
    def __init__(self, dataframe, mfcc_transform):
        self.df = dataframe
        self.transform = mfcc_transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path_str = self.df.iloc[idx]['Ruta']

        try:
            
            paths = []
            if isinstance(path_str, str):
                # Extrae todas las rutas de dentro del WindowsPath('...')
                paths = re.findall(r"WindowsPath\(['\"](.*?)['\"]\)", path_str)
            elif isinstance(path_str, list):
                # Si es una lista (ya en formato correcto)
                paths = [str(p) for p in path_str]

            # Cargar y procesar los archivos de audio
            waveforms = []
            for path in paths:
                waveform, sample_rate = torchaudio.load(path)
                waveforms.append(waveform)

            waveform = torch.mean(torch.stack(waveforms), dim=0)
            mfcc = self.transform(waveform)
            mfcc = mfcc.squeeze(0).transpose(0, 1)

            label = self.df.iloc[idx]['Etiqueta']

        except Exception as e:
            return None

        return {
            'inputs': mfcc,
            'label': torch.tensor(label, dtype=torch.long)
        }


In [4]:
import torchaudio.transforms as T

mfcc_transform = T.MFCC(
    sample_rate=16000,         # según tu config
    n_mfcc=44,                 # según tu config
    melkwargs={
        'n_fft': 400,
        'hop_length': 160,
        'n_mels': 50
    }
)


In [5]:
from torch.utils.data import DataLoader

train_dataset = AudioDataset(train_df, mfcc_transform)


In [6]:
len(train_dataset)

1228

In [7]:
sample = train_dataset[1]
print(sample.keys())         # ['inputs', 'label']
print(sample['inputs'].shape)  # Ej: torch.Size([seq_len, features])
print(sample['label'])         # Ej: tensor(1)

dict_keys(['inputs', 'label'])
torch.Size([903, 44])
tensor(0)


In [8]:
sample['inputs']

tensor([[-4.1874e+01,  6.6053e+01, -5.7372e+00,  ..., -2.6842e+00,
          4.6257e-01,  2.1650e+00],
        [-3.2432e+01,  6.1307e+01, -1.1382e+01,  ..., -2.3972e+00,
         -6.1616e-01, -2.0341e+00],
        [-2.7905e+01,  5.8165e+01, -1.4745e+01,  ..., -2.1649e+00,
         -9.0274e-01, -3.3208e+00],
        ...,
        [-3.0122e+02,  7.3270e+01,  2.4528e+01,  ...,  3.1236e+00,
         -3.3085e-01,  1.3937e+00],
        [-3.1627e+02,  7.5886e+01,  3.5093e+01,  ...,  4.0349e+00,
         -4.0798e+00,  2.1878e-01],
        [-3.2676e+02,  7.1769e+01,  4.1932e+01,  ...,  1.5136e-01,
         -3.6074e-02,  1.8607e-01]])

In [9]:
def collate_fn_pad(batch):
    import torch.nn.functional as F

    # Filtrar elementos inválidos
    batch = [x for x in batch if x is not None and x['inputs'].ndim == 2 and x['inputs'].shape[1] == 44]
    
    if len(batch) == 0:
        print("[SKIP] Batch vacío tras filtrar entradas inválidas.")
        return None

    inputs = []
    labels = []
    for item in batch:
        x = item['inputs']
        y = item['label']
        inputs.append(x)
        labels.append(y)

    # Padding en la dimensión 0 (timesteps)
    max_len = max(input.shape[0] for input in inputs)
    padded_inputs = [F.pad(input, (0, 0, 0, max_len - input.shape[0])) for input in inputs]

    try:
        inputs_tensor = torch.stack(padded_inputs)
        labels_tensor = torch.tensor(labels)
        return {'inputs': inputs_tensor, 'label': labels_tensor}
    except Exception as e:
        print("Error al hacer el stack:", e)
        return None


In [10]:
from torch.utils.data import random_split

# Supongamos que tienes el dataset original como `train_dataset`
train_size = int(0.8 * len(train_dataset))  # 80% para entrenamiento
val_size = len(train_dataset) - train_size  # 20% para validación

# Dividimos el dataset
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

Train dataset size: 982
Validation dataset size: 246


In [11]:
train_dataset[0]

In [12]:
import torch as th
from torch.utils.data import DataLoader
from tqdm import tqdm  # Importar tqdm para la barra de progreso
from sklearn.metrics import accuracy_score, f1_score  # Para calcular métricas

# Asegúrate de que el modelo esté en modo de evaluación
# No lo pongas en eval hasta la parte de validación

# Crear un DataLoader para iterar sobre el dataset de entrenamiento y validación
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn_pad)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn_pad)

# Definir un optimizador y una función de pérdida
optimizer = th.optim.Adam(model.parameters(), lr=0.001)  # Optimización
criterion = th.nn.CrossEntropyLoss()  # Función de pérdida para clasificación multiclase

# Medir el tiempo de cada época
for epoch in range(10):
    
    # Inicializar las listas para las predicciones de esta época
    train_predictions = []
    train_true_labels = []
    val_predictions = []
    val_true_labels = []
    
    # ----- Entrenamiento -----
    model.train()
    train_loader_tqdm = tqdm(train_loader, desc=f"Epoch {epoch+1} - Training", leave=True)
    for batch in train_loader_tqdm:
        if batch is None:
            continue
        inputs = batch['inputs'].to(device)  # Asegúrate de que los datos están en el mismo dispositivo que el modelo
        labels = batch['label'].to(device)   # Para poder comparar después con las etiquetas reales

        # Pasa los datos por el modelo
        outputs = model({'inputs': inputs})

        # Obtener la pérdida
        loss = criterion(outputs, labels)

        # Backpropagation y optimización
        optimizer.zero_grad()  # Resetear los gradientes
        loss.backward()  # Calcular los gradientes
        optimizer.step()  # Actualizar los pesos del modelo

        # Obtener las predicciones (puedes aplicar softmax si deseas probabilidades)
        _, preds = th.max(outputs, 1)  # Si la salida del modelo es logits

        # Almacenar las predicciones y las etiquetas reales
        train_predictions.extend(preds.cpu().numpy())
        train_true_labels.extend(labels.cpu().numpy())
        
        # Actualizar la barra de progreso de entrenamiento
        train_loader_tqdm.set_postfix({'train_batch': len(train_predictions)})

    # ----- Validación -----
    model.eval()
    val_loader_tqdm = tqdm(val_loader, desc=f"Epoch {epoch+1} - Validation", leave=True)
    with th.no_grad():  # No calculamos gradientes para la inferencia
        for batch in val_loader_tqdm:
            if batch is None:
                continue
            inputs = batch['inputs'].to(device)  # Asegúrate de que los datos están en el mismo dispositivo que el modelo
            labels = batch['label'].to(device)   # Para poder comparar después con las etiquetas reales

            # Pasa los datos por el modelo
            outputs = model({'inputs': inputs})

            # Obtener las predicciones (puedes aplicar softmax si deseas probabilidades)
            _, preds = th.max(outputs, 1)  # Si la salida del modelo es logits

            # Almacenar las predicciones y las etiquetas reales
            val_predictions.extend(preds.cpu().numpy())
            val_true_labels.extend(labels.cpu().numpy())
            
            # Actualizar la barra de progreso de validación
            val_loader_tqdm.set_postfix({'val_batch': len(val_predictions)})

    # Calcular y mostrar las métricas después de cada época
    accuracy = accuracy_score(val_true_labels, val_predictions)
    f1 = f1_score(val_true_labels, val_predictions, average='weighted')
    macro_f1 = f1_score(val_true_labels, val_predictions, average='macro')

    print(f"Epoch {epoch+1} completed")
    print(f"Validation Accuracy: {accuracy:.4f}")
    print(f"Validation F1: {f1:.4f}")
    print(f"Validation Macro F1: {macro_f1:.4f}")


c:\Users\Usuario\OneDrive - UPV\Master\HAIA\Trabajo\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Epoch 1 - Validation: 100%|██████████| 8/8 [00:03<00:00,  2.27it/s, val_batch=202]


Epoch 1 completed
Validation Accuracy: 0.6782
Validation F1: 0.5482
Validation Macro F1: 0.1347


Epoch 2 - Validation: 100%|██████████| 8/8 [00:03<00:00,  2.08it/s, val_batch=202]


Epoch 2 completed
Validation Accuracy: 0.6782
Validation F1: 0.5482
Validation Macro F1: 0.1347


Epoch 3 - Validation: 100%|██████████| 8/8 [00:04<00:00,  1.89it/s, val_batch=202]


...

Epoch 8 completed
Validation Accuracy: 0.6485
Validation F1: 0.5685
Validation Macro F1: 0.1886


Epoch 9 - Validation: 100%|██████████| 8/8 [00:03<00:00,  2.29it/s, val_batch=202]


Epoch 9 completed
Validation Accuracy: 0.6584
Validation F1: 0.5541
Validation Macro F1: 0.1601


Epoch 10 - Validation: 100%|██████████| 8/8 [00:04<00:00,  1.89it/s, val_batch=202]

Epoch 10 completed
Validation Accuracy: 0.5891
Validation F1: 0.5312
Validation Macro F1: 0.1612


In [14]:
# Imprimir las primeras predicciones
print(val_predictions[:20])
print(val_true_labels[:20])

[0, 0, 2, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0]
[2, 0, 0, 0, 0, 1, 1, 2, 0, 5, 1, 1, 0, 0, 0, 0, 0, 1, 0, 2]


In [24]:
import torch
from torch.utils.data import Dataset
import torchaudio
import re  # Para usar expresiones regulares

class AudioTestDataset(Dataset):
    def __init__(self, dataframe, mfcc_transform):
        self.df = dataframe
        self.transform = mfcc_transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path_str = self.df.iloc[idx]['Ruta']

        try:
            paths = []
            if isinstance(path_str, str):
                # Extrae todas las rutas de dentro del WindowsPath('...')
                paths = re.findall(r"WindowsPath\(['\"](.*?)['\"]\)", path_str)
            elif isinstance(path_str, list):
                # Si es una lista (ya en formato correcto)
                paths = [str(p) for p in path_str]

            # Cargar y procesar los archivos de audio
            waveforms = []
            for path in paths:
                waveform, sample_rate = torchaudio.load(path)
                waveforms.append(waveform)

            waveform = torch.mean(torch.stack(waveforms), dim=0)
            mfcc = self.transform(waveform)
            mfcc = mfcc.squeeze(0).transpose(0, 1)

        except Exception as e:
            return None

        # Solo devolvemos los 'inputs' para el conjunto de test
        return {
            'inputs': mfcc
        }


In [27]:
def collate_fn_pad_test(batch):
    import torch.nn.functional as F

    # Filtrar elementos inválidos
    batch = [x for x in batch if x is not None and x['inputs'].ndim == 2 and x['inputs'].shape[1] == 44]
    
    if len(batch) == 0:
        print("[SKIP] Batch vacío tras filtrar entradas inválidas.")
        return None

    inputs = []
    for item in batch:
        x = item['inputs']  # Solo necesitamos las entradas para el test
        inputs.append(x)

    # Padding en la dimensión 0 (timesteps)
    max_len = max(input.shape[0] for input in inputs)
    padded_inputs = [F.pad(input, (0, 0, 0, max_len - input.shape[0])) for input in inputs]

    try:
        inputs_tensor = torch.stack(padded_inputs)  # Apilamos las entradas con padding
        return {'inputs': inputs_tensor}  # Devolvemos solo las entradas
    except Exception as e:
        print("Error al hacer el stack:", e)
        return None


In [45]:
# Crear el dataset para test (usando la misma transformación)

test_df = pd.read_csv("test_afc_audio.csv")
test_dataset = AudioTestDataset(test_df, mfcc_transform)

# Crear el DataLoader para el conjunto de test
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn_pad_test)

# Ahora puedes realizar la evaluación sobre el conjunto de test

In [46]:
len(test_df)

2160

In [48]:
import numpy as np

model.eval()
test_predictions = []

# Usamos tqdm para mostrar el progreso
test_loader_tqdm = tqdm(test_loader, desc="Test Evaluation", leave=True)

with th.no_grad():
    for batch in test_loader_tqdm:
        if batch is None or 'inputs' not in batch or len(batch['inputs']) == 0:  # Verifica si el batch está vacío o no tiene 'inputs'
            test_predictions.append(0)  # Usamos append para agregar un único valor

        else:
            inputs = batch['inputs'].to(device)
            outputs = model({'inputs': inputs})
            
            _, preds = torch.max(outputs, 1)
            test_predictions.extend(preds.cpu().numpy())  # extend para agregar múltiples predicciones

# Reemplazamos los NaN con 0 (o con la clase mayoritaria que prefieras)
test_predictions = [0 if np.isnan(pred) else pred for pred in test_predictions]

# Ahora `test_predictions` tiene las predicciones con NaN reemplazados por 0


Test Evaluation:  46%|████▌     | 991/2160 [00:15<00:04, 249.11it/s]

[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch

Test Evaluation:  50%|█████     | 1081/2160 [00:15<00:02, 416.56it/s]

[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch

Test Evaluation:  95%|█████████▍| 2049/2160 [00:29<00:00, 305.14it/s]

[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch

Test Evaluation: 100%|██████████| 2160/2160 [00:29<00:00, 72.47it/s] 

[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch vacío tras filtrar entradas inválidas.
[SKIP] Batch

In [49]:
# Ahora, puedes guardar las predicciones de test o visualizarlas
print("Test Predictions:", test_predictions)

Test Predictions: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 2, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 

In [50]:
len(test_predictions)

2160

In [51]:
df_predictions = pd.DataFrame(test_predictions, columns=["Prediction"])

# Guardar el DataFrame en un archivo CSV
df_predictions.to_csv('BILSTM_audio.csv', index=False)

print("Las predicciones han sido guardadas en 'test_predictions.csv'.")

Las predicciones han sido guardadas en 'test_predictions.csv'.


In [52]:
df_predictions["Prediction"].value_counts()

Prediction
0    1819
1     179
2     162
Name: count, dtype: int64